In [2]:
import pandas as pd
import numpy as np
import itertools

In [3]:
data = pd.read_csv('weather.csv')

In [4]:
data.head()

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,mild,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [5]:
def concateneListe(lst,mot):
    for i in range(len(lst)):
        lst[i]= mot+str(lst[i])
    return lst
def binariser(data):
    datbin = pd.DataFrame()
    incompatible = {}
    for col in data.columns:
        for val in data.loc[:,col].unique():
            datbin[col+str(val)] = (data.loc[:,col]==val)
            incompatible[col+str(val)] = concateneListe(data.loc[:,col].unique().tolist(),col)
    return datbin,incompatible

In [6]:
databin,incompat = binariser(data)
databin.head()

,outlooksunny,outlookovercast,outlookrainy,temperaturehot,temperaturemild,temperaturecool,humidityhigh,humiditynormal,windyFalse,windyTrue,playno,playyes
0,True,False,False,True,False,False,True,False,True,False,True,False
1,True,False,False,True,False,False,True,False,False,True,True,False
2,False,True,False,False,True,False,True,False,True,False,False,True
3,False,False,True,False,True,False,True,False,True,False,False,True
4,False,False,True,False,False,True,False,True,True,False,False,True


In [7]:
def support(databin,itemset):
    arr = np.array(len(databin)*[True])
    for item in itemset:
        arr = np.logical_and(arr,databin.loc[:,item])
    return arr.sum()/len(databin)
def confiance(databin,A,B):
    return support(databin,A+B)/support(databin,A)
def lift(databin,A,B):
    return support(databin,A+B)/(support(databin,A)*support(databin,B))

In [8]:
def product(left,databin):
    t = itertools.product(left,databin.columns.tolist())
    res = []
    for i in t:
        res = res + [i[0]+[i[1]]]
    return res

In [9]:
def prodCart(databin,incompat,left,deg):
    b = False
    res = []
    for i in product(left,databin):
        for d in range(deg-1):
            b = False
            temp = i[d]
            i = list(i)
            i[d] = ''
            if(len(list(set(i) & set(incompat[temp])))!=0):
                b = True
                break;
            i[d] = temp
        if(not b):
            if(set(i) not in res):
                res = res + [set(i)]
    return res

In [10]:
def fermeture(databin,itemset):
    arr = np.array(len(databin)*[True])
    for item in itemset:
        arr = np.logical_and(arr,databin.loc[:,item])
    ferm = []
    for item in databin.columns:
        if(np.logical_and(arr,databin.loc[:,item]).sum()==arr.sum()):
            ferm = ferm + [item]
    return set(ferm)

In [11]:
def CLOSE(data,minsup):
    databin,incompat = binariser(data)
    #Initialisation
    candidats = []
    for item in databin.columns:
        if(support(databin,[item])>=minsup):
            candidats = candidats + [([item],support(databin,[item]),fermeture(databin,[item]))]
    #La boucle
    ferm = []
    left = []
    regles = []
    deg = 2
    while(len(candidats)!=0):
        cand = []
        while(len(candidats)!=0):
            c = candidats.pop(0)
            s = support(databin,c[0])
            if(s>=minsup):
                left = left + [c[0]]
                c = list(c)
                c[1] = s; c[2]=fermeture(databin,c[0])
                ferm = ferm + [c[2]]
                c = tuple(c)
                cand = cand + [c]
        regles = regles + cand
        #Iteration suivante
        print("iteration")
        for i in prodCart(databin,incompat,left,deg):
            if(i not in ferm):
                candidats = candidats + [(list(i),0,[])]
        deg = deg + 1
        left = []
    return(regles)

In [12]:
CLOSE(data,0.1)

iteration
iteration
iteration
iteration


[(['outlooksunny'], 0.35714285714285715, {'outlooksunny'}),
 (['outlookovercast'], 0.2857142857142857, {'outlookovercast', 'playyes'}),
 (['outlookrainy'], 0.35714285714285715, {'outlookrainy'}),
 (['temperaturehot'],
  0.14285714285714285,
  {'humidityhigh', 'outlooksunny', 'playno', 'temperaturehot'}),
 (['temperaturemild'], 0.5714285714285714, {'temperaturemild'}),
 (['temperaturecool'],
  0.2857142857142857,
  {'humiditynormal', 'temperaturecool'}),
 (['humidityhigh'], 0.5, {'humidityhigh'}),
 (['humiditynormal'], 0.5, {'humiditynormal'}),
 (['windyFalse'], 0.5714285714285714, {'windyFalse'}),
 (['windyTrue'], 0.42857142857142855, {'windyTrue'}),
 (['playno'], 0.35714285714285715, {'playno'}),
 (['playyes'], 0.6428571428571429, {'playyes'}),
 (['outlooksunny', 'temperaturehot'],
  0.14285714285714285,
  {'humidityhigh', 'outlooksunny', 'playno', 'temperaturehot'}),
 (['outlooksunny', 'temperaturemild'],
  0.14285714285714285,
  {'outlooksunny', 'temperaturemild'}),
 (['outlooksunny